<div align="right"><em>Машинне навчання. Лабораторний практикум</em></div>

# Лабораторна робота №6
## Аналіз тональності тексту: CountVectorizer & TF-IDF*
*на прикладі IMDB оглядів фільмів


---


## Завдання

Необхідно:
- розібратися з особливостями роботи з текстовими даними на етапі виділення ознак (Bag of words, TF-IDF);
- побудувати модель класифікації оглядів (відгуків) фільмів на основі тональності відповідних текстів;
- заповнити пропущений код. 

## Виконання завдання

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Зчитуємо дані з файлу в пам'ять у вигляді об'єкта `Pandas.DataFrame`.

In [2]:
imdb_data = pd.read_csv('../data/IMDB_dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Виведемо сумарну інформацію

In [3]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
imdb_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Бачимо, що набір даних збалансований.

### Попередня робота з текстом

Почистимо текстові дані. Видалимо HTML теги та інший шумовий текст.

In [5]:
# Видаляємо HTML теги
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Видаляємо квадратні дужки
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Видаляємо шумовий текст
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Застосовуємо функцію до стовпця review
imdb_data['review'] = imdb_data['review'].apply(denoise_text)

<>:8: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_38687/1916276498.py:8: SyntaxWarning: invalid escape sequence '\['
  return re.sub('\[[^]]*\]', '', text)


Видалимо спеціальні символи

In [6]:
# Визначимо функцію для видалення спеціальних символів
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern,'',text)
    return text

# Застосовуємо функцію до стовпця review
imdb_data['review'] = imdb_data['review'].apply(remove_special_characters)

**0. Видаліть стоп-слова.** 

In [11]:
from nltk.corpus import stopwords 

from nltk.tokenize import word_tokenize

# Завантажуємо необхідні ресурси NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

# Видаляємо стоп-слова
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    # Ваш код тут
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Застосовуємо функцію до стовпця review
# Ваш код тут
imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alllpina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/alllpina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Проведемо стемінг тексту

In [12]:
# Стемінг
def simple_stemmer(text):
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# Застосовуємо функцію до стовпця review
imdb_data['review'] = imdb_data['review'].apply(simple_stemmer)

### Формування навчальної та відкладеної вибірок

Бінаризуємо мітки класів

In [13]:
lb = LabelBinarizer()
sentiment_data = lb.fit_transform(imdb_data['sentiment'])
sentiment_data.shape

(50000, 1)

**1. Розділіть дані на навчальну і відкладену частини у відношенні 4:1, використовуючи `sklearn.model_selection.train_test_split` з `random_state=17`.**

In [14]:
train_reviews, holdout_reviews, train_sentiments, holdout_sentiments = train_test_split(imdb_data['review'], sentiment_data, test_size=0.2, random_state=17) # Ваш код тут

### Bag of words

**2. Застосуйте модель Bag of words для перетворення текстових документів у числові вектори, використовуючи `CountVectorizer` з `min_df=0`, `max_df=1` та `ngram_range=(1,3)`.**

In [15]:
cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 3)) # Ваш код тут
cv_train_reviews = cv.fit_transform(train_reviews)# Ваш код тут
cv_holdout_reviews = cv.transform(holdout_reviews)# Ваш код тут

print('BOW_cv_train:', cv_train_reviews.shape)
print('BOW_cv_houldout:', cv_holdout_reviews.shape)

BOW_cv_train: (40000, 6829651)
BOW_cv_houldout: (10000, 6829651)


### TF-IDF

**3. Перетворіть текстові документи у матрицю ознак tfidf використовуючи `TfidfVectorizer` з `min_df=0`, `max_df=1`, `ngram_range=(1,3)` та `use_idf=True`.**

In [16]:
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,3), use_idf=True) # Ваш код тут
tv_train_reviews = tv.fit_transform(train_reviews)# Ваш код тут
tv_holdout_reviews = tv.transform(holdout_reviews)# Ваш код тут

print('Tfidf_train:', tv_train_reviews.shape)
print('Tfidf_test:', tv_holdout_reviews.shape)

Tfidf_train: (40000, 6829651)
Tfidf_test: (10000, 6829651)


### Побудова моделі логістичної регресії на CountVectorizer

**4. Навчіть модель логістичної регресії на наборі даних (cv_train_reviews, train_sentiments) з параметрами `penalty='l2'`, `max_iter=100`, `C=1`. Оцініть її якість.**

In [17]:
# Ваш код тут
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(penalty='l2', max_iter=100, C=1)

model.fit(cv_train_reviews, train_sentiments)

predictions = model.predict(cv_holdout_reviews)

accuracy = accuracy_score(holdout_sentiments, predictions)
print(f'Accuracy: {accuracy:.4f}')

print(classification_report(holdout_sentiments, predictions))

/mnt/hdd/University/Machine_Learning/MLVenv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8980
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      4939
           1       0.89      0.91      0.90      5061

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



### Побудова моделі логістичної регресії на TF-IDF ознаках

**5. Навчіть модель логістичної регресії на наборі даних (tv_train_reviews, train_sentiments). Оцініть її якість та порівняйте з попереднім результатом.**

In [18]:
# Ваш код тут
model = LogisticRegression(penalty='l2', max_iter=100, C=1)

model.fit(tv_train_reviews, train_sentiments)

predictions = model.predict(tv_holdout_reviews)

accuracy = accuracy_score(holdout_sentiments, predictions)
print(f'Accuracy: {accuracy:.4f}')

print(classification_report(holdout_sentiments, predictions))

/mnt/hdd/University/Machine_Learning/MLVenv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8885
              precision    recall  f1-score   support

           0       0.89      0.88      0.89      4939
           1       0.88      0.90      0.89      5061

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

